## Concise Logistic Regression for Image Classification

- Shows a concise implementation of logistic regression for image classification
- Uses PyTorch

In [1]:
# imports
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# download the data (uncomment if to download the data locally)
#!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
#!unzip hymenoptera_data.zip

In [2]:
# create data loaders

data_dir = 'hymenoptera_data'

# custom transformer to flatten the image tensors
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        result = torch.reshape(img, self.new_size)      # 重新调整张量
        return result

# transformations used to standardize and normalize the datasets        # 数据集变换
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),         # 将输入的图像大小调整为224x224像素
        transforms.CenterCrop(224),     # 从中心位置裁剪出224x224大小的图像
        transforms.ToTensor(),          # 将图像转换为PyTorch张量的格式
        ReshapeTransform((-1,))         # 将图像张量进行重塑操作，将其形状调整为一维向量
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ReshapeTransform((-1,))
    ]),
}

# load the correspoding folders  创建图像数据集对象
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),        # 指定图像文件夹位置
                                          data_transforms[x])
                  for x in ['train', 'val']}

# load the entire dataset; we are not using minibatches here
train_dataset = torch.utils.data.DataLoader(image_datasets['train'],
                                            batch_size=len(image_datasets['train']),        # 将整个训练集作为一个批次加载
                                            shuffle=True)       # 在每个训练迭代中，数据顺序将被打乱

test_dataset = torch.utils.data.DataLoader(image_datasets['val'],
                                           batch_size=len(image_datasets['val']),
                                           shuffle=True)

In [3]:
# build the LR model
class LR(nn.Module):            # 继承自nn.Module
    def __init__(self, dim):                # 构造函数
        super(LR, self).__init__()
        self.linear = nn.Linear(dim, 1)     # 线性层对象
        nn.init.zeros_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)
        # 使用零初始化权重和偏置项，将线性层的权重和偏置项初始化为零

    def forward(self, x):       # 前向传播
        x = self.linear(x)
        x = torch.sigmoid(x)        # 将线性变换的结果映射到范围为0到1之间的概率值
        return x 

In [4]:
# predict function
def predict(yhat, y):
    yhat = yhat.squeeze()       # 去除维度为1的维度
    y = y.unsqueeze(0)          # 在y的第0个维度上增加一个维度，以便与yhat的维度匹配
    y_prediction = torch.zeros(y.size()[1])     # 创建了一个与y相同大小的全零张量
    for i in range(yhat.shape[0]):      # 分类
        if yhat[i] <= 0.5:
            y_prediction[i] = 0
        else:
            y_prediction[i] = 1
    return 100 - torch.mean(torch.abs(y_prediction - y)) * 100      # 准确率

In [5]:
# model config
dim = train_dataset.dataset[0][0].shape[0]      # 获取训练集中第一个样本的输入特征的形状，然后取其维度大小

lrmodel = LR(dim).to(device)                    # 创建了一个名为lrmodel的逻辑回归模型对象LR(dim)，并将其移动到指定的设备device上
criterion = nn.BCELoss()                        # 创建了一个二分类交叉熵损失函数(nn.BCELoss())对象criterion，用于计算模型预测值与真实标签之间的损失
optimizer = torch.optim.SGD(lrmodel.parameters(), lr=0.0001)            # 创建了一个随机梯度下降（SGD）优化器对象optimizer，用于更新模型的参数

In [8]:
# training the model
costs = []      # 空列表，存储每一次迭代的损失值

for ITER in range(100):         # 100次迭代训练
    lrmodel.train()             # 设置为训练模式
    x, y = next(iter(train_dataset))
    test_x, test_y = next(iter(test_dataset))
    # 获取训练集和测试集的一个批次数据

    # forward
    yhat = lrmodel.forward(x.to(device))        # 使用逻辑回归模型进行前向传播，得到预测值yhat

    cost = criterion(yhat.squeeze(), y.type(torch.FloatTensor).to(device))      # 计算当前预测值与真实标签的损失
    train_pred = predict(yhat, y)       # 使用之前定义的predict函数计算训练集的准确率

    # backward
    optimizer.zero_grad()       # 将优化器的梯度缓冲区清零
    cost.backward()             # 进行反向传播，计算梯度
    optimizer.step()            # 根据梯度更新模型的参数
    
    # evaluate
    lrmodel.eval()          # 评估模式      
    with torch.no_grad():           # 创建一个上下文环境，在该环境中不会计算梯度，以提高评估的效率
        yhat_test = lrmodel.forward(test_x.to(device))      # 使用逻辑回归模型进行前向传播，得到测试集的预测值
        test_pred = predict(yhat_test, test_y)              # 使用之前定义的predict函数计算测试集的准确率

    if ITER % 10 == 0:
        costs.append(cost)          # 将当前损失cost添加到costs列表中

    if ITER % 10 == 0:
        print("Cost after iteration {}: {} | Train Acc: {} | Test Acc: {}".format(ITER, 
                                                                                    cost, 
                                                                                    train_pred,
                                                                                    test_pred))
        # 输出当前迭代的损失值cost、训练集的准确率train_pred和测试集的准确率test_pred
   

Cost after iteration 0: 0.6931471228599548 | Train Acc: 50.40983581542969 | Test Acc: 45.75163269042969
Cost after iteration 10: 0.6691471338272095 | Train Acc: 64.3442611694336 | Test Acc: 54.24836730957031
Cost after iteration 20: 0.6513182520866394 | Train Acc: 68.44261932373047 | Test Acc: 54.24836730957031
Cost after iteration 30: 0.6367825269699097 | Train Acc: 68.03278350830078 | Test Acc: 54.24836730957031
Cost after iteration 40: 0.6245337128639221 | Train Acc: 69.67213439941406 | Test Acc: 54.90196228027344
Cost after iteration 50: 0.6139225363731384 | Train Acc: 70.90164184570312 | Test Acc: 56.20914840698242
Cost after iteration 60: 0.6045235395431519 | Train Acc: 72.54098510742188 | Test Acc: 56.86274337768555
Cost after iteration 70: 0.5960512161254883 | Train Acc: 74.18032836914062 | Test Acc: 57.51633834838867
Cost after iteration 80: 0.5883084535598755 | Train Acc: 73.77049255371094 | Test Acc: 57.51633834838867
Cost after iteration 90: 0.5811557769775391 | Train Acc: 

### References
- [A Logistic Regression Model from Scratch](https://colab.research.google.com/drive/1iBoJ0kngkOthy7SgVaVQA1aHEROt5mra?usp=sharing)